2.	Напишите программный код в виде функции, реализующий проверку описанной в пособии гипотезы о стрельбе. Параметры гипотезы и ее проверки (число испытаний, число промахов, вероятность промаха по нулевой гипотезе, по альтернативной гипотезе, уровень значимости и желаемая мощность проверки) сделайте вводимыми параметрами.

In [ ]:
from math import comb
#Вероятность получить k успехов
def kf_binomial(n, k, p):
    return comb(n, k) * (p**k) * ((1 - p) ** (n - k)) #Реализация биномиального коефициента

#Эксперимент
def test(n, k,  p0 = 0.1, alpha = 0.05):
    #Вероятность наблюдаемого
    p_obs = kf_binomial(n, k, p0)
    #Суума всехисходов эксперимента
    probs = [kf_binomial(n, i, p0) for i in range(n + 1)]
    p_test = sum(p for p in probs if p <= p_obs + 1e-15)

    print(f"Нулевая гипотеза: p = {p0}")
    print(f"Число промахов: {k} из {n}")
    print(f"p_test = {p_test:.3f}")

    if p_test < alpha:
        print("Отвергаем")
    else:
        print("Нет оснований отвергать")
    return p_test

test(n = 100, k = 12, p0=0.2, alpha=0.05)

Определите предельное число промахов (сверху и снизу) для заданного уровня значимости. Как это число зависит от уровня значимости? Почему так?

In [ ]:
from math import comb

#Вероятность получить k промахов
def binom(n, k, p):
    return comb(n, k) * (p**k) * ((1 - p)**(n - k))

#Вероятность получить не болбьше k промахов
def ctf(n, k, p):
    return sum(binom(n, i, p) for i in range(0, k + 1))

#Поиск критических границ
#низняя граница
def find_critical_bonus(n, p0 = 0.2, alpha = 0.05):
    k_low = 0
    while ctf(n, k_low, p0) < alpha / 2:
        k_low += 1
    
    #Верхняя граница
    k_high = n
    while(1 - ctf(n, k_high - 1, p0)) < alpha / 2:
        k_high -= 1
    return k_low, k_high

#Запуск
n = 100
p0 = 0.2
for alpha in [0.1, 0.05, 0.01]:
    low, high = find_critical_bonus(n, p0, alpha)
    print(f"alpha = {alpha}: границы [{low}, {high}]")

4.	Определите мощность проверки для заданных параметров нулевой и альтернативной гипотез. Как можно влиять на мощность проверки?

In [ ]:
from math import comb

#Биномиальный коефициент

def binom_pmf(n, k, p):
    
    return comb(n, k) * (p**k) * ((1 - p)**(n - k))

def binom_cdf(n, k, p):
   
    return sum(binom_pmf(n, i, p) for i in range(0, k+1))

#Критические границы

def critical_bounds(n, p0, alpha=0.05):
   
    # Нижняя граница: вероятность слишком малого числа промахов
    k_low = 0
    while binom_cdf(n, k_low, p0) < alpha/2:
        k_low += 1

    # Верхняя граница: вероятность слишком большого числа промахов
    k_high = n
    while (1 - binom_cdf(n, k_high-1, p0)) < alpha/2:
        k_high -= 1

    return k_low, k_high

# Мощность теста

def test_power(n, p0, p1, alpha=0.05):

    # Находим критическую область
    k_low, k_high = critical_bounds(n, p0, alpha)

    # Вероятность попасть в критическую область при p = p1
    power_low = binom_cdf(n, k_low, p1)          # слишком мало промахов
    power_high = 1 - binom_cdf(n, k_high-1, p1)  # слишком много промахов
    power = power_low + power_high

    return k_low, k_high, power


#Запуск
n = 100     # число выстрелов
p0 = 0.2    # вероятность промаха по нулевой гипотезе
p1 = 0.25   # вероятность промаха по альтернативе
alpha = 0.05  # уровень значимости

k_low, k_high, power = test_power(n, p0, p1, alpha)

print(f"Нулевая гипотеза: p0 = {p0}")
print(f"Альтернативная гипотеза: p1 = {p1}")
print(f"Критическая область: X ≤ {k_low} или X ≥ {k_high}")
print(f"Мощность теста при n={n}, alpha={alpha}: {power:.3f}")


5.	Задана числовая последовательность 1.7, -5.4, -4.0, -5.9, -1.6, 0.0, 0.6, 2.1, 0.1, -4.9, -3.5, 5.9, 8.5, 9.9, 13.3, 11.1, 14.4, 16.2. Проверьте гипотезу о том, что последовательность аппроксимируется экспоненциальной функцией по уровню значимости 5%. 

In [ ]:
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

#Данные по условию задачи
y = np.array([1.7, -5.4, -4.0, -5.9, -1.6, 0.0, 0.6, 2.1, 0.1,
              -4.9, -3.5, 5.9, 8.5, 9.9, 13.3, 11.1, 14.4, 16.2])

x = np.arange(1, len(y) + 1)
#Сдвиг вверх, чтобы все стали положительными
shift = abs(min(y)) + 1
y_shifted = y + shift

ln_y = np.log(y_shifted)

#линейная регрессия
X = sm.add_constant(x)
model = sm.OLS(ln_y, X).fit()
print(model.summary())

#Апроксимация
a = np.exp(model.params[0]) / np.exp(0 * model.params[1])
b = model.params[1]
y_fit = a * np.exp(b * x) - shift

#график
plt.scatter(x, y, label = "Данные")
plt.plot(x, y_fit, 'r', label = "Экспотенцияальная апроксимация")
plt.legend()
plt.show()